### Задачи
1. Выделить участок +-100,000 нуклеотидов до и после транскрипта
2. Проверить какие гены лежат в аннотации в этом участке
3. Предсказание промотеров с помощью - CNNPromoter_e, DeePromoter, PromID, Depicter
4. Визуализировать участок на хромосоме (https://github.com/WenchaoLin/chrVis)

In [21]:
## Выделяем все участки найденные в геноме


top_mmseq_results= '../tables/transcripts_top_uniq_t2t.tsv'
transript_4_matches = []

with open(top_mmseq_results) as fh:
    for line in fh:
        line = line.strip().split('\t')
        if line[0].startswith('4'):
            transript_4_matches.append(line)
            
            
## функция читает фаста-файл и возвращает только сиквенс который находится под нужным хедером
def fasta_read_by_header(path_to_fasta_file, header):
    "Returns dicttionary with header:sequence"
    was_header = False
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(f">{header}"):
                was_header = True
                seq = []
            elif line.startswith('>') and was_header:
                return ''.join(seq)
            elif was_header:
                seq.append(line)
            

## выделяем все сиквенсы матчей 4 транскрипта += 10,000 нуклеотидов
match_to_seq = {}
for i in transript_4_matches:
    header = i[1]
    start = int(i[9]) - 1000
    stop  = int(i[10]) + 1000
    key = f"trans_4_chr_{i[2]}_len_{i[4]}_{start}_{stop}"
    match_to_seq[key] = fasta_read_by_header(
        '/home/dzilov/data_zilov/data/modal_genomes/human_t2t_CHM13v2_0/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna',
        header
    )[start-1:stop]

In [23]:
### записываем в fasta 
file = '../data/transcript_4_1000bp.fasta'

with open(file, 'w') as fw:
    for key, value in match_to_seq.items():
        line_to_write = f'>{key}\n{value.upper()}\n'
        fw.write(line_to_write)

### Установил и запустил depicter, он показывает лучший результат определения промотеров по данным статьи https://doi.org/10.1093/bib/bbab551
1. mamba create -n depicter -c bioconda -c conda-forge -c defaults h5py=2.9.0 keras=2.2.4 numpy=1.16.5 pandas=0.25.1 scikit-learn=0.21.3 scipy=1.3.1 tensorflow=1.13.1
2. conda activate depicter
3. cd ~/data_zilov/soft
4. git clone https://github.com/zhuyaner/Depicter.git
5. python ~/data_zilov/soft/Depicter/Depicter_pred.py --input ../data/transcript_4_1000bp.fasta --output transript_4_promoters --species Human --seq_type full_length --type TATA+_TATA-

Тул не работает, с ним все плохо, пытался переписать поломанные части но все тщетно

### Визуализация транскриптов на хромосомах

In [27]:
## считаем длины хромасом
def fasta_reader_yield(path_to_fasta_file):
    header = None
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    yield header,"".join(seq)
                header = line[1:]
                seq = []
            else:
                seq.append(line)
        if header:
            yield header,"".join(seq)

chr_to_length = {}
for i in fasta_reader_yield('/home/dzilov/data_zilov/data/modal_genomes/human_t2t_CHM13v2_0/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna'):
    chr_to_length[i[0].split()[0]] = len(i[1])

In [31]:
from collections import defaultdict

refseq_to_chm_number = {}

with open("../../chm_to_acc_numbers.tsv") as fh:
    for line in fh:
        if line.startswith('#'):
            continue
        line = line.strip().split("\t")
        refseq_to_chm_number[line[6]] = line[0]
        
#transcript

uniqs = defaultdict(list)
uniqs_info = defaultdict(list)

## m8 == outftmt6 
## query_seq_id => target_seq_id => chm_number => identity => length => mismatch => gaps => 
## ==> start_query => stop_query => start_target => stop_target => e_value => bit-score
with open("../mmseq_search_results/result_t2t_transcript.m8") as fh:
    for line in fh:
        line = line.strip().split("\t")
        if line[1] not in uniqs[line[0]]:
            uniqs[line[0]].append(line[1])
            line = line[:2] + [refseq_to_chm_number[line[1]]] + line[2:]
            uniqs_info[line[0]].append(line)

In [40]:
### Создаем конфиг файл с длинами хромосом
chr_config = '../chrViz_config/chr_config.txt'

with open(chr_config, 'w') as fw:
    for chm_acc, length in chr_to_length.items():
        chm_n = refseq_to_chm_number[chm_acc]
        line_to_write = f'chr{chm_n}\tchr{chm_n}\t{length}\t#333300\n'
        fw.write(line_to_write)

### Создаем конфиг файл с позициями транскриптов и их цветами на рисунке

colors = {'1': '#bacac6', '2': '#00bc12', '3': '#f35336', '4': '#be002f'}

trans_config = '../chrViz_config/trans_config.txt'

with open(trans_config, 'w') as fw:
    for match, infos in uniqs_info.items():
        for info in infos:
            chm_n = info[2]
            trans_n = info[0][0][0]
            start = info[-4]
            stop = info[-3]
            line_to_write = f'chr{chm_n}\ttrans_{trans_n}\t{start}\t{stop}\t{colors[trans_n]}\n'
            fw.write(line_to_write)